In [58]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
ames = pd.read_csv("/content/drive/My Drive/AmesHousing.csv")

In [53]:
ames.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [66]:
lr = LinearRegression()

X = ames[["Gr Liv Area", "TotRms AbvGrd", "Bldg Type", "Lot Area"]]
y = ames["SalePrice"]



X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train_s = (X_train - X_train.mean())/X_train.std()

<ipython-input-66-278abf2bd01c>:10: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_train_s = (X_train - X_train.mean())/X_train.std()
<ipython-input-66-278abf2bd01c>:10: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_train_s = (X_train - X_train.mean())/X_train.std()


In [98]:
# Model using size and number of rooms
pipeline1 = Pipeline([
    ('scaling', StandardScaler()),
    ('regression', LinearRegression())
]).set_output(transform="pandas")

# Fit the pipeline
pipeline1.fit(X_train[['Gr Liv Area', 'TotRms AbvGrd']], y_train)

# Prediction
prediction1 = pipeline1.predict(X_test[['Gr Liv Area', 'TotRms AbvGrd']])
mean_prediction1 = np.mean(prediction1)
mean_prediction1

181497.15102244

In [68]:
# RMSE
rmse1 = mean_squared_error(y_test, prediction1, squared=False)
rmse1

55181.64867967372

In [95]:
# Model using size, number of rooms, and building type
pipeline2 = Pipeline([
    ('preprocess', ColumnTransformer([
        ('num', StandardScaler(), ['Gr Liv Area', 'TotRms AbvGrd']),
        ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])
    ],
    remainder = "drop")),
    ('regression', LinearRegression())

])

# Fit the pipeline
pipeline2.fit(X_train[['Gr Liv Area', 'TotRms AbvGrd', 'Bldg Type']], y_train)

# Predictions
prediction2 = pipeline2.predict(X_test[['Gr Liv Area', 'TotRms AbvGrd', 'Bldg Type']])
mean_prediction1 = np.mean(prediction2)
mean_prediction1

181887.56996837378

In [72]:
# Compute RMSE
rmse2 = mean_squared_error(y_test, prediction2, squared=False)
rmse2

53948.518328767976

In [77]:
pipeline3 = Pipeline([
    ('preprocess', ColumnTransformer([
        ('num', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True), ['Gr Liv Area']),
        ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])
    ])),
    ('regression', LinearRegression())
])

# Fit the pipeline
pipeline3.fit(X_train[['Gr Liv Area', 'Bldg Type']], y_train)

# Predictions
prediction3 = pipeline3.predict(X_test[['Gr Liv Area', 'Bldg Type']])
mean_prediction3 = np.mean(prediction2)
mean_prediction3

181887.56996837378

In [78]:
# RMSE
rmse3 = mean_squared_error(y_test, prediction3, squared=False)
rmse3

53978.04082526511

In [80]:
pipeline4 = Pipeline([
    ('preprocess', ColumnTransformer([
        ('size_poly', PolynomialFeatures(degree=5), ['Gr Liv Area']),
        ('rooms_poly', PolynomialFeatures(degree=5), ['TotRms AbvGrd']),
        ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])
    ])),
    ('regression', LinearRegression())
])

# Fit the pipeline
pipeline4.fit(X_train[['Gr Liv Area', 'TotRms AbvGrd', 'Bldg Type']], y_train)

# Predictions
prediction4 = pipeline4.predict(X_test[['Gr Liv Area', 'TotRms AbvGrd', 'Bldg Type']])
mean_prediction4 = np.mean(prediction2)
mean_prediction4

181887.56996837378

In [81]:
# Compute RMSE
rmse4 = mean_squared_error(y_test, prediction4, squared=False)
rmse4

55189.15300425339

The best model based on RMSE is the model using size, number of rooms, and building type because its RMSE is the lowest.

In [110]:
from sklearn.model_selection import cross_val_score
scoresrmse_scorer = make_scorer(mean_squared_error, squared=False)
scores = cross_val_score(pipeline1, X, y, cv=5, scoring=rmse_scorer)
scores

ValueError: ignored

In [111]:
from sklearn.metrics import make_scorer, mean_squared_error
rmse_scorer = make_scorer(mean_squared_error, squared=False)
scores = cross_val_score(pipeline2, X, y, cv=5, scoring=rmse_scorer)
scores

array([59447.5945622 , 51677.04316677, 57660.55257913, 54423.45405505,
       47631.76278281])

In [112]:
scores.mean()

54168.081429193844

In [113]:
rmse_scorer = make_scorer(mean_squared_error, squared=False)
scores = cross_val_score(pipeline3, X, y, cv=5, scoring=rmse_scorer)
scores

array([59148.73696256, 52141.93955263, 57470.44206793, 54616.25407089,
       48345.40142339])

In [114]:
scores.mean()

54344.55481548086

In [115]:
rmse_scorer = make_scorer(mean_squared_error, squared=False)
scores = cross_val_score(pipeline4, X, y, cv=5, scoring=rmse_scorer)
scores

array([61412.86775346, 53487.90359056, 62977.41324647, 55348.05645308,
       48289.98483857])

In [116]:
scores.mean()

56303.24517642801

Though the first model didn't work, I would say with a good amount of certainty that the second model performs the best with the lowest RMSE.

In [126]:
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('size_poly', PolynomialFeatures(), ['Gr Liv Area']),
        ('rooms_poly', PolynomialFeatures(), ['TotRms AbvGrd']),
        ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])
    ])),
    ('regression', LinearRegression())
])


param_grid = {
    'preprocessor__size_poly__degree': range(1, 11),
    'preprocessor__rooms_poly__degree': range(1, 11),
}

# Set up the grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Perform the grid search
gscv_fitted = grid_search.fit(X, y)
gscv_fitted.cv_results_

{'mean_fit_time': array([0.08781662, 0.04799461, 0.03945022, 0.04657331, 0.03632364,
        0.04174514, 0.02701254, 0.05437651, 0.05215526, 0.03347497,
        0.02323318, 0.0162653 , 0.01554446, 0.01569967, 0.01989183,
        0.01780372, 0.0264327 , 0.02106667, 0.03000307, 0.03590412,
        0.02599683, 0.01636505, 0.01685953, 0.01715202, 0.01751227,
        0.01788239, 0.04173884, 0.02123346, 0.02432775, 0.0257206 ,
        0.017941  , 0.01749058, 0.01732545, 0.01648021, 0.02021642,
        0.03719273, 0.02194796, 0.02573109, 0.02460485, 0.02955384,
        0.02493329, 0.01728821, 0.0162241 , 0.02923574, 0.02065592,
        0.03925071, 0.02700696, 0.02760706, 0.0264442 , 0.02662234,
        0.02077699, 0.01709361, 0.03573861, 0.02464185, 0.02536297,
        0.02168097, 0.03085456, 0.0367517 , 0.0349236 , 0.02994981,
        0.01800251, 0.04642563, 0.04366241, 0.05687132, 0.06990705,
        0.06617966, 0.05617094, 0.0683424 , 0.06563735, 0.06684375,
        0.05664206, 0.0563777 ,

In [127]:
gscv_fitted.cv_results_['mean_test_score']

array([-2.95199396e+09, -2.92175413e+09, -2.79802096e+09, -2.84490990e+09,
       -3.43303001e+09, -4.12239324e+09, -5.93838068e+09, -1.21074296e+10,
       -3.49276483e+10, -1.10417773e+11, -2.95776263e+09, -2.94657669e+09,
       -2.80530745e+09, -2.84387625e+09, -3.43303018e+09, -4.12239324e+09,
       -5.93838068e+09, -1.21074358e+10, -3.49276352e+10, -1.10417755e+11,
       -2.93244233e+09, -2.94118072e+09, -2.82269740e+09, -2.84315944e+09,
       -3.12001321e+09, -4.12239324e+09, -5.93838074e+09, -1.21074341e+10,
       -3.49276430e+10, -1.10417651e+11, -2.89181129e+09, -2.92746336e+09,
       -2.84066759e+09, -2.80276479e+09, -3.17246821e+09, -4.12239324e+09,
       -5.93838059e+09, -1.21074320e+10, -3.49276509e+10, -1.10417778e+11,
       -2.89418084e+09, -2.96019306e+09, -2.86465615e+09, -2.80472722e+09,
       -3.19879614e+09, -4.12239325e+09, -5.93838059e+09, -1.21074320e+10,
       -3.49276509e+10, -1.10417778e+11, -2.93650960e+09, -2.93980364e+09,
       -2.87504737e+09, -

In [131]:
df = pd.DataFrame(data = {
    "degrees": np.arange(1, 101),
    "scores": gscv_fitted.cv_results_['mean_test_score']
})

sorted_df = df.sort_values(by="scores", ascending=True)

sorted_df = sorted_df.reset_index(drop=True)

sorted_df

,degrees,scores
0,93,-2.707163e+12
1,92,-1.298924e+12
2,91,-1.263105e+12
3,100,-1.104192e+11
4,90,-1.104192e+11
...,...,...
95,64,-2.819038e+09
96,13,-2.805307e+09
97,44,-2.804727e+09
98,34,-2.802765e+09


Q1: Which model performed the best?

It appears that the 93rd model - 93 degree  - had the best cross-validated metrics.

Q2: What downsides do you see of trying all possible model options? How might you go about choosing a smaller number of tuning values to try?

Downsides are computational cost/time, risk of overfitting, and complexity issues.  Things like regularization and bayesian optimization may be ways to go about choosing smaller numbers.